## Cleaning, Merging, and Initial Feature Engineering

In [26]:
# Import libraries 
import pandas as pd

#### Step 1: Load in datasets

In [27]:
spray = pd.read_csv('assets/input/spray.csv')
weather = pd.read_csv('assets/input/weather.csv')
west_nile = pd.read_csv('assets/input/train.csv')
test = pd.read_csv('assets/input/test.csv')

#### Step 2: Clean weather dataset

In [28]:
# Investigate weather
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [29]:
# Inspect number of rows and columns
weather.shape

(2944, 22)

The first step is to examine and handle missing data from weather. According to the data dictionary from NOAA, M represents missing data and T represents trace amounts in the applicable columns. 

In [30]:
# Identify number of missing rows per column
weather[weather=='M'].count()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise           0
Sunset            0
CodeSum           0
Depth          1472
Water1         2944
SnowFall       1472
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [31]:
# Water1 appears to have the same number of missing rows are total number of rows
weather['Water1'].unique()

array(['M'], dtype=object)

In [32]:
# Water1 column appears to only have missing values, so we'll drop it
weather = weather.drop('Water1', axis=1)

Next, we'll take a look at all the columns with a substantial number of M's (>1000), as indicated above. This includes Depart, Depth, and SnowFall. We will decide to either keep or drop these columns. 

In [33]:
# Summary of Depart, Depth, and SnowFall column values 
weather[['Depart', 'Depth', 'SnowFall']].describe()

,Depart,Depth,SnowFall
count,2944,2944,2944
unique,42,2,4
top,M,M,M
freq,1472,1472,1472


In [34]:
# Half of the depth data is either missing or 0
weather['Depth'].value_counts()

M    1472
0    1472
Name: Depth, dtype: int64

In [35]:
# 50% of the snowfall values are missing - likely one weather station does not record snowfall 
weather['SnowFall'].value_counts(normalize=True)['M']

0.5

In [37]:
# Drop depth and snowfall columns since they don't appear to have enough useful information
# Drop depart column, as we suspect that absolute temperatures are more important than relative temperatures 
weather.drop(['Depth', 'SnowFall', 'Depart'], axis=1, inplace = True)

Now that we have dropped columns with a significant amount of missing data, we will procede by only cleaning columns that we think we be useful for modeling. We are keeping in mind the theory that West Nile is more likely in hot and dry weather than cold and wet. 

In [38]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb',
       'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [39]:
# Define features by our perception of importance - want to quanitfy temp, humidity, wind, and precipitation
good_features = ['Station', 'Date', 'Tavg', 'WetBulb', 'PrecipTotal','AvgSpeed']
maybe_features = ['Heat','Cool']

In [40]:
# Investigate good features for further cleaning
weather[good_features].head()

,Station,Date,Tavg,WetBulb,PrecipTotal,AvgSpeed
0,1,2007-05-01,67,56,0.00,9.2
1,2,2007-05-01,68,57,0.00,9.6
2,1,2007-05-02,51,47,0.00,13.4
3,2,2007-05-02,52,47,0.00,13.4
4,1,2007-05-03,56,48,0.00,11.9


We need to strip spaces from our columns and impute values to 'M' and 'T'. Note,'T' only appears in PrecipTotal. For trace amounts in PrecipTotal, we assume a value of 0.005, which is below the minimum recorded positive amount.  There were only 2 observations of 'M' in PrecipTotal, we assume they are 0.

In [41]:
# Define dataframe that will be used as a mask to update weather later
# Remove spaces before T and M
stripped_weather = weather[good_features].select_dtypes('object').apply(lambda x: x.str.strip())

In [42]:
# Replace PrecipTotal T and M values 
stripped_weather['PrecipTotal'].replace("T",'0.005',inplace=True)
stripped_weather['PrecipTotal'].replace("M", '0.00',inplace=True)

In [43]:
# Update weather dataframe with new values 
weather['PrecipTotal'] = stripped_weather['PrecipTotal']

In [44]:
# Identify remaining missing values in the features of interest
weather[good_features][weather[good_features] == 'M'].count()

Station         0
Date            0
Tavg           11
WetBulb         4
PrecipTotal     0
AvgSpeed        3
dtype: int64

Since there are so few missing values in AvgSpeed and WetBulb, we will drop those rows. We can infer the Tavg from averaging Tmin and Tmax.

In [45]:
# Identify row index and mean values to update Tavg
avgs = weather[['Tmax', 'Tmin']][weather['Tavg']=='M'].mean(axis=1)
weather.loc[avgs.index,'Tavg'] = avgs

In [47]:
# Drop missing rows from avgspeed and wetbulb
mask = (weather['WetBulb'] == 'M') | (weather['AvgSpeed']=='M')
weather.drop(weather.index[mask], inplace=True)

In [49]:
# Convert object dtypes into float
cols = ['Station','Tavg', 'WetBulb', 'PrecipTotal', 'AvgSpeed']
weather[cols] = weather[cols].astype(float)

Todo: feature engineering on weather's good_features